In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp src

# Core functions

## load data

In [ ]:
#| export
import pandas as pd

In [ ]:
#| export
def load_data(fp='../in/utterances_requests_50k_labeled.csv'):
	df = pd.read_csv('../in/utterances_requests_50k_labeled.csv')
	# in the `label` column, replace `1.0` with `1` and `NaN` with `0`
	df["label"] = df["label"].fillna(0).astype(int)
	return df